In [1]:
#Ensure we are running Python 3
import sys
pyinfo = sys.version_info
assert (pyinfo.major == 3), "Python 3 required. The version you are using is: {}".format(pyinfo) 

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

In [515]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def colorspace(img, c_space):
    return cv2.cvtColor(img, c_space)

def get_channel(img, c):
    return img[:, :, c]

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def drawROI(img, x1, y1, x2, y2):
    out = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0))
    return out

def drawPolygon(img):
    pts = np.array([[200,450],[740,450],[1080,720],[200,720]], np.int32)
    '''p1 = [0.45*1280, 0.6*720]
    p2 = [0.55*1280, 0.6*720]
    p3= [0.90*1280, 0.96*720]
    p4= [0.10*1280, 0.96*720]'''
    pts = np.array([p1, p2, p3, p4], dtype=np.int32)
    pts = pts.reshape((-1,1,2))
    out = cv2.polylines(img,[pts],True,(0,255,255))
    return out

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

run your solution on all test_images and make copies into the test_images directory).

In [487]:
#Adaptive Binary Threshold.
#We use R and G channels from RGB, V from HSV and S from HLS.
#We iteratively find threshold parameters that give enough pixels but not too many

def get_ROI(img, x1, y1, x2, y2):
    return img[y1:y2, x1:x2, :]

#Binary Thresholding on on ROI image. 
# input ROI image is 3 channel
# returns a 3channel Binary image (0 and 255)
def binary_threshold(roi, initSV = 140, initR = 140, thresh_dt = 5, bailout = 20):
    
    minSV = 140
    maxSV = 240
    minR = 140
    maxR = 250 
    
    SVThresh = initSV
    RThresh = initR
    
    thresh_img = np.zeros_like(roi[:, :, 0])
    
    total_pixels = thresh_img.shape[0]*thresh_img.shape[1]
    #thresh = init_thresh
    
    #Using R and G channels to mask out dark or grey road/ shadow values
    (B,G,R) = cv2.split(roi)
    
    R_dx = np.absolute(cv2.Sobel(R, cv2.CV_64F, 1, 0))
    R_dx *= 255/np.max(R_dx)
    
    hls = colorspace(roi, cv2.COLOR_BGR2HLS)
    S = get_channel(hls, 2)
    
    hsv = colorspace(roi, cv2.COLOR_BGR2HSV)
    V = get_channel(hsv, 2)
    
    #Create an initial binary image. We will refine this if we don't have enough pixels or too many pixels
    thresh_img[(V >= SVThresh) | (R >= RThresh)] = 255# | (S >= SVThresh) ] = 255
    #thresh_img[(R < minRG) & (G < minRG)] = 0
    
    #Count of Non-Zero mask pixels
    nzcount = np.count_nonzero(thresh_img)
    
    
    #Bailout Counter. If we can not reach a good value in n steps, stop wasting time and move on.
    counter = 0
    
    
    b = np.zeros_like(thresh_img)
    g = np.zeros_like(thresh_img)
    r = np.zeros_like(thresh_img)
    
    wiggleScope = True
    #We want the number of lane pixels to be within 1% to 3% of the total pixels in image
    minarea =0.015
    maxarea =0.02
    while ((nzcount < minarea*total_pixels) | (nzcount >= maxarea*total_pixels)) & (wiggleScope):
        counter += 1
        if (counter == bailout):
            #print("Unable to find a good value in {} steps. Bailing out!".format(bailout))
            nzcount = np.count_nonzero(thresh_img)
            thresh_img = np.zeros_like(thresh_img)
            SVThresh = initSV
            RThresh = initR
            break
            
        #If there is still scope in moving ranges
        VwiggleScope = (SVThresh >= minSV) & (SVThresh <= maxSV)
        RwiggleScope = (RThresh >= minR) & (RThresh <= maxR)
        wiggleScope = VwiggleScope | RwiggleScope
        
        if wiggleScope:
            if (nzcount < minarea*total_pixels):
                if VwiggleScope:
                    SVThresh -= thresh_dt
                if RwiggleScope:
                    RThresh -= thresh_dt
            else:
                if VwiggleScope:
                    SVThresh += thresh_dt
                if RwiggleScope:
                    RThresh += thresh_dt
                    
            thresh_img = np.zeros_like(thresh_img)
            thresh_img[(V >= SVThresh) | (R >= RThresh)] = 255# | (S >= SVThresh) ] = 255
            #thresh_img[(V > thresh) | ((R > thresh) & (G > thresh)) | (S > thresh) ] = 255
            #thresh_img[(R < minRG) & (G < minRG)] = 0
            nzcount = np.count_nonzero(thresh_img)
            
        else:
            #print("Unable to find a good value in range. Bailing out!")
            nzcount = np.count_nonzero(thresh_img)
            thresh_img = np.zeros_like(thresh_img)
            SVThresh = initSV
            RThresh = initR
            break
    
    
    print("%cnt", nzcount/total_pixels)
    print("steps", counter)    
    print(nzcount, total_pixels)
        
    #b[(V >= SVThresh)] = 255
    #r[(R >= RThresh)] = 255
    #g[(S >= SVThresh)] = 255
    #print(r.shape, thresh_img.shape)
    #print(SVThresh, RThresh)
    bin_img = np.dstack((thresh_img, thresh_img, thresh_img))
    #bin_img = np.dstack((r,g,b))
    return bin_img, SVThresh, RThresh

#read saved calibration and distortion



In [565]:
#Perspective warping...These values work for the test image sequences
#Warping from the ROI to Bird Eye View. Creating a small image coz it saves time. large image probably
#doesn't give any thing extra
p1 = [380, 10]#for full ROI warp 
p2 = [510, 10]#for full ROI warp
#p1 = [350, 30] #for a little lesser...cleaner, but shorter data
#p2 = [540, 30] #for a little lesser...cleaner, but shorter data
p3= [875, 235]
p4= [45, 235]
src_pts = np.array([p1, p2, p3, p4], dtype=np.float32)
offset = 100
dst_pts = np.array([[offset, 0], [640-offset, 0], [640-offset,240], [offset, 240]], dtype=np.float32)
M = cv2.getPerspectiveTransform(src_pts, dst_pts)


def warp(img, M):
    out = cv2.warpPerspective(img, M, (640, 240))
    return out

In [566]:
"""
Run the processing pipeline on image
if return_at == -1, all steps are perfomed, else, steps are performed till the required number
"""
def processImage(image, return_at = -1):
    
    Y,X,C = image.shape
    lines = None
    
    #undistort image
    #dst_img = cv2.undistort(image, mtx, dist, None, mtx)
    
    #extract image ROI
    #Most parts of the image do not provide useful info
    #roi = drawROI(image, *(200, 400), *(1080, 720)) 
    ROI_x1, ROI_y1, ROI_x2, ROI_y2 = 200, 450, 1080, 720 
    roi = get_ROI(image, ROI_x1, ROI_y1, ROI_x2, ROI_y2)
    #plt.imshow(roi)
    #plt.show()
    out_image = image
    bin_img,sv,r = binary_threshold(roi)
    #plt.imshow(bin_img)
    #plt.show()
    #weighted_image = cv2.addWeighted(bin_img, 0.8, roi, 0.2, 0)
    
    out_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    out_image[ROI_y1:ROI_y2, ROI_x1:ROI_x2, :] = bin_img
   
    #out_image = drawPolygon(out_image)
    
    
    warped_image = warp(bin_img, M)
    out_image[0:240, -640:] = warped_image
    return lines, out_image

In [567]:
img_dir = "test_images"
for img_name in os.listdir(img_dir):
    #ONly look at image files:
    if (img_name):# ==  "test5.jpg"):
        if os.path.isfile(os.path.join("test_images", img_name)):
            print(img_name)
            img_in_path = os.path.join("test_images", img_name);

            image = mpimg.imread(img_in_path)
            lines, out_image = processImage(image, -1)
            #mpimg.imsave(img_out_path, out_image)
            img_name = os.path.splitext(os.path.basename(img_name))[0]
            img_out_path = os.path.join("test_images", "output", img_name);
            cv2.imwrite(img_out_path+".png", out_image)
        
print("Done")

challenge_1.jpg
%cnt 0.015875420875420876
steps 6
3772 237600
test1.jpg
%cnt 0.01999158249158249
steps 18
4750 237600
test3.jpg
%cnt 0.019886363636363636
steps 16
4725 237600
challenge_3.jpg
%cnt 0.017344276094276093
steps 6
4121 237600
straight_lines2.jpg
%cnt 0.019894781144781146
steps 17
4727 237600
test2.jpg
%cnt 0.019457070707070707
steps 5
4623 237600
test4.jpg
%cnt 0.01715909090909091
steps 17
4077 237600
challenge_4.jpg
%cnt 0.015593434343434343
steps 5
3705 237600
challenge_5.jpg
%cnt 0.01539983164983165
steps 11
3659 237600
test6.jpg
%cnt 0.01925084175084175
steps 13
4574 237600
straight_lines1.jpg
%cnt 0.018872053872053873
steps 13
4484 237600
test5.jpg
%cnt 0.019343434343434343
steps 14
4596 237600
challenge_2.jpg
%cnt 0.019482323232323233
steps 6
4629 237600
Done


In [501]:
out_image[(out_image>0) & (out_image<255)]

array([147, 147, 147,  72,  72,  72,  16,  16,  16,  53,  53,  53,  19,
        19,  19,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,
        56,  56,  56,  56, 159, 159, 159, 181, 181, 181,  33,  33,  33,
        56,  56,  56, 132, 132, 132, 193, 193, 193,  56,  56,  56, 199,
       199, 199, 201, 201, 201, 106, 106, 106, 187, 187, 187, 151, 151,
       151,   5,   5,   5,  14,  14,  14,  13,  13,  13, 117, 117, 117,
        94,  94,  94,  94,  94,  94,  56,  56,  56, 182, 182, 182,   9,
         9,   9,  48,  48,  48,  53,  53,  53,   9,   9,   9, 145, 145,
       145, 149, 149, 149,  75,  75,  75,  42,  42,  42, 135, 135, 135,
        68,  68,  68, 128, 128, 128, 106, 106, 106, 201, 201, 201,  80,
        80,  80,  50,  50,  50, 250, 250, 250,  74,  74,  74,   1,   1,
         1,  13,  13,  13,  45,  45,  45, 207, 207, 207, 162, 162, 162,
       230, 230, 230,  54,  54,  54,  71,  71,  71, 247, 247, 247, 191,
       191, 191, 135, 135, 135, 104, 104, 104,  76,  76,  76,   

In [453]:
# Import everything needed to edit/save/watch video clips
import imageio.plugins
imageio.plugins.ffmpeg.download()
import moviepy.editor as mp
from moviepy.editor import VideoFileClip
from IPython.display import HTML



In [454]:
def process_image(image, return_at=-1):#(return_at, image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    _, result = processImage(image, return_at)
    return result

In [455]:
import tqdm
white_output = 'project_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_out.mp4
[MoviePy] Writing video project_out.mp4


100%|█████████▉| 1260/1261 [00:53<00:00, 23.54it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_out.mp4 

CPU times: user 5min 51s, sys: 3 s, total: 5min 54s
Wall time: 54.1 s


In [456]:
yellow_output = 'challenge_out.mp4'
clip2 = VideoFileClip('challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video challenge_out.mp4
[MoviePy] Writing video challenge_out.mp4


100%|██████████| 485/485 [00:20<00:00, 23.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_out.mp4 

CPU times: user 2min 17s, sys: 1.25 s, total: 2min 18s
Wall time: 21.5 s


In [457]:
yellow_output = 'harder_out.mp4'
clip2 = VideoFileClip('harder_challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video harder_out.mp4
[MoviePy] Writing video harder_out.mp4


100%|█████████▉| 1199/1200 [01:02<00:00, 19.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_out.mp4 

CPU times: user 6min 46s, sys: 3.07 s, total: 6min 49s
Wall time: 1min 3s
